In [4]:
#@title Clone the repo
!git clone https://github.com/Graylab/GeoDock.git
%cd /content/GeoDock

Cloning into 'GeoDock'...
remote: Enumerating objects: 354, done.
remote: Counting objects: 100% (60/60), done.
remote: Compressing objects: 100% (45/45), done.
remote: Total 354 (delta 25), reused 36 (delta 15), pack-reused 294
Receiving objects: 100% (354/354), 354.59 MiB | 35.95 MiB/s, done.
Resolving deltas: 100% (123/123), done.
Updating files: 100% (56/56), done.
/content/GeoDock


In [5]:
#@title Hit `Runtime` -> `Run all`, then input docking partner1 and partner2
from google.colab import files
out_name = 'test' #@param {type:"string"}
upload1 = files.upload()
upload2 = files.upload()
partner1 = list(upload1.keys())[0]
partner2 = list(upload2.keys())[0]

Saving 1AVX_l_u.pdb to 1AVX_l_u.pdb


Saving 1AVX_r_u.pdb to 1AVX_r_u.pdb


In [8]:
#@title Install dependencies
%%time
%%capture
import sys
import torch
pyt_version_str=torch.__version__.split("+")[0].replace(".", "")
version_str="".join([
    f"py3{sys.version_info.minor}_cu",
    torch.version.cuda.replace(".",""),
    f"_pyt{pyt_version_str}"
])
!pip install fvcore iopath
!pip install --no-index --no-cache-dir pytorch3d -f https://dl.fbaipublicfiles.com/pytorch3d/packaging/wheels/{version_str}/download.html

def format_pytorch_version(version):
  return version.split('+')[0]

TORCH_version = torch.__version__
TORCH = format_pytorch_version(TORCH_version)

def format_cuda_version(version):
  return 'cu' + version.replace('.', '')

CUDA_version = torch.version.cuda
CUDA = format_cuda_version(CUDA_version)

!pip install torch-scatter     -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-sparse      -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-cluster     -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-spline-conv -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-geometric
!pip install pytorch-lightning
!pip install py3Dmol
!pip install fair-esm
!pip install biotite
!pip install einops

CPU times: user 240 ms, sys: 35.6 ms, total: 276 ms
Wall time: 36.5 s


In [15]:
#@title Run Prediction
from geodock.GeoDockRunner import GeoDockRunner
torch.cuda.empty_cache()
ckpt_file = "geodock/weights/db5_bound.ckpt"
geodock = GeoDockRunner(ckpt_file=ckpt_file)
pred = geodock.dock(
    partner1=partner1,
    partner2=partner2,
    out_name=out_name
)

Downloading: "https://dl.fbaipublicfiles.com/fair-esm/models/esm2_t33_650M_UR50D.pt" to /root/.cache/torch/hub/checkpoints/esm2_t33_650M_UR50D.pt
Downloading: "https://dl.fbaipublicfiles.com/fair-esm/regression/esm2_t33_650M_UR50D-contact-regression.pt" to /root/.cache/torch/hub/checkpoints/esm2_t33_650M_UR50D-contact-regression.pt
INFO:pytorch_lightning.utilities.migration.utils:Lightning automatically upgraded your loaded checkpoint from v1.9.5 to v2.0.4. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint --file geodock/weights/db5_bound.ckpt`


Completed embedding in 3.59 seconds.
File './test.pdb' does not exist.
Completed docking in 1.71 seconds.


In [16]:
#@title Display 3D structure {run: "auto"}
import py3Dmol

color = "chain" #@param ["chain", "lDDT", "rainbow"]

view = py3Dmol.view(js='https://3dmol.org/build/3Dmol.js',)
view.addModel(open(out_name+'.pdb','r').read(),'pdb')
if color == 'chain':
  view.setStyle({'chain': 'A'},{'cartoon': {'color': 'green'}})
  view.setStyle({'chain': 'B'},{'cartoon': {'color': 'blue'}})
elif color == 'lDDT':
  view.setStyle({'cartoon': {'colorscheme': {'prop':'b','gradient': 'roygb','min':50,'max':90}}})
elif color == 'rainbow':
  view.setStyle({'model': -1}, {"cartoon": {'color': 'spectrum'}})
view.zoomTo()
view.show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol